## Approach for searching the contexts
- For each pdf in pdf_names:
  - First construct contexts dictionary with contexts as keys and doc_id, block_id, sentence_number, **all extractions' linear_order_number** from {this_pdf_name}_events.xlsx
  - Get doc_sentence_map with doc_id, sentence_text, sentence_id, doc_sent_linear_order from {this_pdf_name}_all.csv
  - For each context key in contexts dictionary :
    - this_doc_id, sentence_number from contexts dictionary
    - from doc_sentence_map, get all sentences up until **doc_sent_map's doc_sent_linear_order**
    - get 3 top closest doc_id, sentence_number pairs where the context was found up until this_doc_id,this_sentence_number
    - calculate distance between **doc_sent_map's doc_sent_linear_order** between context dictionary value and doc_sentence_map for each of the nearest context matches
- Plot the histogram

## Approach for ordering the documents, extractions
- For each pdf in pdf_names:
  - First sort doc_id, block_id, sentence_number and add linear_order_number from {this_pdf_name}_all_linear_order.xlsx
  - Construct doc_sentence_map with doc_id, sentence_text, sentence_id, and add doc_sent_linear_order from {this_pdf_name}_all.csv
  - For each of the sorted doc_id, block_id, sentence_number and linear_order_number, add doc_sent_linear_order.
  - Now only save events from this list doc_id, block_id, sentence_number and linear_order_number and doc_sent_linear_order along with contexts and sentence text {this_pdf_name}_events.xlsx
  - This is because mentions["documents"] and mentions["extractions"] do not match. mentions["extractions"] have page numbers and block numbers. But again COSMOS JSON does not have block numbers. So we need to sort in this order doc_id, block_id, sentence_number and add linear_order_number for extractions, then a linear order number for documents after sorting doc_ids and sentence_numbers. And combine the two for searching the contexts within documents by document linear order.

In [1]:
!pip install altair
!pip install altair vega_datasets
!pip install vega
!pip install altair_viewer
!pip install textwrap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.2/190.2 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.9/84.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 48.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 844.5/844.5 kB 26.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple

In [2]:
import numpy as np
import pandas as pd
import os
import json
import io

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!ls
%cd "/content/drive/MyDrive/Colab Notebooks/skema/data"
!ls

drive  sample_data
/content/drive/MyDrive/Colab Notebooks/skema/data
'1471-2334-3-19 (2) (2).pdf'
 cosmos-and-extractions-jsons-for-3-papers
 data_modeling_covid_italy.json
 data-response-to-covid-19-was-italy-unprepared.json
 data-sars-double.json
 doc_pg_blk_sent_event.xlsx
 event_linear_order_modeling_covid_italy.xlsx
 event_linear_order_modeling.xlsx
 event_linear_order_response_to_covid_19_was_italy_unprepared.xlsx
 event_linear_order_sarsdouble.xlsx
'modeling_covid_italy_all (1).gsheet'
 modeling_covid_italy_all.csv
 modeling_covid_italy_all.gsheet
 modeling_covid_italy_all.json
 modeling_covid_italy_all_linear_order_5_17.xlsx
 modeling_covid_italy_all_linear_order.xlsx
 modeling_covid_italy_all.xlsx
'modeling_covid_italy_events (1).gsheet'
 modeling_covid_italy_events.csv
 modeling_covid_italy_events.gsheet
 modeling_covid_italy_events.xlsx
 modeling_covid_italy.json
 modeling_covid_italy.xlsx
'modelling_doc_event (1).xlsx'
 modelling_doc_event.xlsx
 response-to-covid-19-was-ita

In [5]:
path = "/content/drive/MyDrive/Colab Notebooks/skema/data/"
os.path.exists(path)

True

In [6]:
filenames = ["sarsdouble.xlsx", "modeling_covid_italy.xlsx", "response-to-covid-19-was-italy-unprepared.xlsx"]
diff_distance_map = []
for filename in filenames:
  print("\n**********************************************************************************\n")
  print("The annotated extractions file name %s " %(filename))
  df = pd.read_csv(os.path.join(path, filename))
  df['locationContext'] = df['locationContext'].replace({"^'|'$": ""}, regex=True)
  df['temporalContext'] = df['temporalContext'].replace({"^'|'$": ""}, regex=True)

  print("Number of page numbers different from extractions with that of manually annotated are : %d out of %d" %(len(df[df['pg_num'] != df["page_num"]]), df.shape[0]) )
  gp = df.groupby(['locationContext']).count()
  print(gp.reset_index()[['locationContext', 'event_id']])


**********************************************************************************

The annotated extractions file name sarsdouble.xlsx 
Number of page numbers different from extractions with that of manually annotated are : 11 out of 172
                                      locationContext  event_id
0                                             Beijing        17
1                                     China,Hong Kong        15
2                    China,Hong Kong,Beijing,Shanghai         1
3     China,Hong Kong,Mainland China,Beijing,Shanghai         1
4             Europe,Inner Mongolia,Hong Kong,Beijing         1
5   Guangdong Province,Hong Kong,China,Mainland Ch...         2
6   Guangdong Province,Hong Kong,Mainland China,Be...         2
7                           Guangdong,China,Hong Kong         1
8                                           Hong Kong        43
9                              Hong Kong,Amoy Gardens        21
10                                Hong Kong,Guangdong   

In [7]:
print("Distances between annotated page numbers cosmos page numbers: ", len(df[df['pg_num'] != df["page_num"]]))

Distances between annotated page numbers cosmos page numbers:  5


In [8]:
path = "/content/drive/MyDrive/Colab Notebooks/skema/data/"
extractions_path = "cosmos-and-extractions-jsons-for-3-papers"
annotated_files = ["data-sars-double.json", "data_modeling_covid_italy.json", "data-response-to-covid-19-was-italy-unprepared.json"]
extraction_files =["extractions_sarsdouble.json", "extractions_modeling_covid_italy--COSMOS-data.json","extractions_response-to-covid-19-was-italy-unprepared--COSMOS-data.json"]
paper_names = ["sarsdouble.pdf", "modeling_covid_italy.pdf", "response-to-covid-19-was-italy-unprepared.pdf"]
ann_extr_file_pairs = {}
for name,ann, extr in zip(paper_names,annotated_files, extraction_files):
  ann_extr_file_pairs[name] = [ann, extr]

In [9]:
def save_extr_ann_file(path, filename, map):
  pd.DataFrame.from_records(map).to_csv(os.path.join(path, filename+".xlsx"))

  with io.open(os.path.join(path, filename+".json"), 'w', encoding='utf-8') as f:
    f.write(json.dumps(map, ensure_ascii=False))

In [10]:
def combine_ann_extr_all(path, extractions_path, extr, ann,filename ):
  with open(os.path.join(path,extractions_path, extr ), "r", encoding='UTF-8') as f:
    contents = f.readlines()
    extractions = json.loads(contents[0])
  with open(os.path.join(path, ann ), "r", encoding='UTF-8') as f:
    contents = f.read()
    annotations = json.loads(contents)

  print(len(annotations), len(extractions["mentions"]), len(extractions["documents"]))
  doc_sentence_map = {}
  linear_order = 1
  doc_ids = sorted(list(extractions["documents"].keys()))
  for doc_id in doc_ids:
    #print(doc_id, document)
    document = extractions["documents"][doc_id]
    for i,sentence in enumerate(document['sentences']):
      doc_sentence_map[(doc_id,i)] = {"sentence_text":sentence['words'], "sno":linear_order}
      linear_order += 1

  doc_event_map = []
  event_doc_map = {}
  for mention in extractions['mentions']:
    # if mention['id'].startswith("E:"):
    for att in mention["attachments"]:
      if "pageNum" in att.keys():
        this_text = doc_sentence_map[(mention['document'], mention['sentence'])]['sentence_text']
        this_linear_order = doc_sentence_map[(mention['document'], mention['sentence'])]['sno']

        doc_event_map.append({"doc_id":mention['document'], "pg_num":att["pageNum"][0],"blk_id":att["blockIdx"][0],"sentence_id":mention['sentence'], 
                              "doc_sentence_count":len(extractions["documents"][mention['document']]["sentences"]), 
                              "event_id":mention["id"], "event":mention["text"] }) #corrected_sent_number -> 1,2,5,4,6,7, => 1,2,3,4,5,6
        event_doc_map[mention["id"]] = {"doc_id":mention['document'],"pg_num":att["pageNum"][0],"blk_id":att["blockIdx"][0],"sentence_id":mention['sentence'], 
                              "doc_sentence_count":len(extractions["documents"][mention['document']]["sentences"]), 
                              "event_id":mention["id"], "event":mention["text"] , "sentence_text":this_text, "doc_sent_linear_order":this_linear_order}
  event_ann_map = {}
  for ann in annotations:
    event_ann_map[ann['eventId']] = {"annotated_page_num":ann["page_num"],"para_num":ann["para_num"], "event":ann["event"], 'locationContext': ann['locationContext'],
    'temporalContext': ann['temporalContext'],'explanation': ann['explanation']}
  
  empty_map = {"annotated_page_num":"","para_num":"", "event":"", 'locationContext': "",
    'temporalContext': "",'explanation': ""}
  event_extr_ann_map = []
  for event, values in event_doc_map.items():
    this_event = event_ann_map[event] if event in event_ann_map.keys() else empty_map
    event_extr_ann_map.append({"doc_id":values['doc_id'],"annotated_page_num":this_event["annotated_page_num"],"para_num":this_event["para_num"], "event_id":values["event_id"],
                               "event":this_event["event"], 'locationContext': ",".join(this_event['locationContext']),
    "sentence_text":",".join(values["sentence_text"]),
    'temporalContext': ",".join(this_event['temporalContext']),'explanation': this_event['explanation'], 'pg_num':values['pg_num'], 'blk_id':values['blk_id'], 
    'sentence_id':values['sentence_id'], 'doc_sentence_count':values['doc_sentence_count'], "doc_sent_linear_order":values["doc_sent_linear_order"]})


  df = pd.DataFrame.from_records(event_extr_ann_map)
  df = df[df.columns]
  print(df.columns.to_list())
  print("\n")
  df['locationContext'] = df['locationContext'].replace({"^'|'$": ""}, regex=True)
  df['temporalContext'] = df['temporalContext'].replace({"^'|'$": ""}, regex=True)
  df.sort_values(by=['doc_id', 'pg_num','blk_id', 'sentence_id'], inplace=True,
               ascending = [True, True, True, True])
  d = df[['doc_id', 'pg_num','blk_id','sentence_id', 'doc_sentence_count']]
  df['linear_order'] = [i for i in range( 1,len(d)+1, 1)]
  print("columns \n",df.columns)
  df.to_csv(os.path.join(path, filename+"_linear_order_5_17"+".xlsx"))
  save_extr_ann_file(path, filename, event_extr_ann_map)
  

### Uncomment following lines to combine annotations and extractions

In [11]:
for key in paper_names:
  name = key
  ann, extr = ann_extr_file_pairs[name]
  combine_ann_extr_all(path, extractions_path, extr, ann, name[:-4]+"_all")

174 6212 91
['doc_id', 'annotated_page_num', 'para_num', 'event_id', 'event', 'locationContext', 'sentence_text', 'temporalContext', 'explanation', 'pg_num', 'blk_id', 'sentence_id', 'doc_sentence_count', 'doc_sent_linear_order']


columns 
 Index(['doc_id', 'annotated_page_num', 'para_num', 'event_id', 'event',
       'locationContext', 'sentence_text', 'temporalContext', 'explanation',
       'pg_num', 'blk_id', 'sentence_id', 'doc_sentence_count',
       'doc_sent_linear_order', 'linear_order'],
      dtype='object')
302 10045 76
['doc_id', 'annotated_page_num', 'para_num', 'event_id', 'event', 'locationContext', 'sentence_text', 'temporalContext', 'explanation', 'pg_num', 'blk_id', 'sentence_id', 'doc_sentence_count', 'doc_sent_linear_order']


columns 
 Index(['doc_id', 'annotated_page_num', 'para_num', 'event_id', 'event',
       'locationContext', 'sentence_text', 'temporalContext', 'explanation',
       'pg_num', 'blk_id', 'sentence_id', 'doc_sentence_count',
       'doc_sent_l

In [12]:
def get_doc_id_sentence_text(path, extractions_path, extr,filename ):
  with open(os.path.join(path,extractions_path, extr ), "r", encoding='UTF-8') as f:
    contents = f.readlines()
    extractions = json.loads(contents[0])

  print(len(extractions["documents"]))
  doc_sentence_map = []
  linear_order = 1
  doc_ids = sorted(list(extractions["documents"].keys()))
  for doc_id in doc_ids:
    #print(doc_id, document)
    document = extractions["documents"][doc_id]
    for i,sentence in enumerate(document['sentences']):
      doc_sentence_map.append({"doc_id":doc_id,"sentence_id":i,"sentence_text":" ".join(sentence['words']),
                               "raw_text":" ".join(sentence['raw']), "text":document['text'], "sno":linear_order})
      if len(sentence['words']) != len(sentence['raw']):
        print("words and raw words lengths are not equal!")
      linear_order += 1
  pd.DataFrame.from_records(doc_sentence_map).to_csv(os.path.join(path, filename+".csv"))

In [13]:
for key in paper_names:
  name = key
  ann, extr = ann_extr_file_pairs[name]
  get_doc_id_sentence_text(path, extractions_path, extr, name[:-4]+"_all")

91
76
47


In [14]:
filenames = ["sarsdouble_all.xlsx", "modeling_covid_italy_all.xlsx", "response-to-covid-19-was-italy-unprepared_all.xlsx"]

### Save all event extractions with linear orders

In [15]:
filenames = ["sarsdouble_all_linear_order_5_17.xlsx", "modeling_covid_italy_all_linear_order_5_17.xlsx", "response-to-covid-19-was-italy-unprepared_all_linear_order_5_17.xlsx"]
diff_distance_map = []
for filename in filenames:
  print("\n**********************************************************************************\n")
  print("The annotated extractions file name %s " %(filename))
  df = pd.read_csv(os.path.join(path, filename), index_col=False)
  df = df[df.columns[1:]]
  print(df.columns.to_list())
  print("\n")
  event_df = df[df['event_id'].str.startswith("E:")]
  print(len(event_df), len(df))
  f = filename.replace("-","_").split("_")
  f = "_".join(f[:f.index("all")])
  print(f,f+"_event.xlsx")
  event_df.sort_values(by=['doc_id', 'pg_num','blk_id', 'sentence_id', 'linear_order'], inplace=True,
               ascending = [True, True, True, True, True])
  event_df.to_csv(os.path.join(path, f+"_events.csv"))
  


**********************************************************************************

The annotated extractions file name sarsdouble_all_linear_order_5_17.xlsx 
['doc_id', 'annotated_page_num', 'para_num', 'event_id', 'event', 'locationContext', 'sentence_text', 'temporalContext', 'explanation', 'pg_num', 'blk_id', 'sentence_id', 'doc_sentence_count', 'doc_sent_linear_order', 'linear_order']


172 6108
sarsdouble sarsdouble_event.xlsx


<ipython-input-15-8d23addc7c87>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_df.sort_values(by=['doc_id', 'pg_num','blk_id', 'sentence_id', 'linear_order'], inplace=True,



**********************************************************************************

The annotated extractions file name modeling_covid_italy_all_linear_order_5_17.xlsx 
['doc_id', 'annotated_page_num', 'para_num', 'event_id', 'event', 'locationContext', 'sentence_text', 'temporalContext', 'explanation', 'pg_num', 'blk_id', 'sentence_id', 'doc_sentence_count', 'doc_sent_linear_order', 'linear_order']


302 10045
modeling_covid_italy modeling_covid_italy_event.xlsx


<ipython-input-15-8d23addc7c87>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_df.sort_values(by=['doc_id', 'pg_num','blk_id', 'sentence_id', 'linear_order'], inplace=True,



**********************************************************************************

The annotated extractions file name response-to-covid-19-was-italy-unprepared_all_linear_order_5_17.xlsx 
['doc_id', 'annotated_page_num', 'para_num', 'event_id', 'event', 'locationContext', 'sentence_text', 'temporalContext', 'explanation', 'pg_num', 'blk_id', 'sentence_id', 'doc_sentence_count', 'doc_sent_linear_order', 'linear_order']


42 4429
response_to_covid_19_was_italy_unprepared response_to_covid_19_was_italy_unprepared_event.xlsx


<ipython-input-15-8d23addc7c87>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_df.sort_values(by=['doc_id', 'pg_num','blk_id', 'sentence_id', 'linear_order'], inplace=True,


In [16]:
event_df[['doc_id','pg_num', 'blk_id','sentence_id', 'doc_sentence_count', 'linear_order']]

,doc_id,pg_num,blk_id,sentence_id,doc_sentence_count,linear_order
1916,-1949274406,2,4,3,13,1917
2102,-1949274406,2,4,10,13,2103
2156,-1949274406,2,4,12,13,2157
1519,-1810507189,8,2,1,15,1520
1588,-1810507189,8,2,5,15,1589
1618,-1810507189,8,2,8,15,1619
1680,-1810507189,8,2,11,15,1681
1198,-1675118546,11,2,1,9,1199
1213,-1675118546,11,2,2,9,1214
1294,-1675118546,11,2,6,9,1295


In [17]:
from collections import Counter

filenames = ["sarsdouble_all_linear_order_5_17.xlsx", "modeling_covid_italy_all_linear_order_5_17.xlsx", "response-to-covid-19-was-italy-unprepared_all_linear_order_5_17.xlsx"]
loc_context_counter = Counter()
temporal_context_counter = Counter()
loc_context_counters = {filename: Counter() for filename in filenames}
temporal_context_counters = {filename: Counter() for filename in filenames}

loc_tf = {}
with pd.ExcelWriter(os.path.join(path, "summarized_context_counter.xlsx")) as writer:

  for filename in filenames:
    print("\n**********************************************************************************\n")
    print("The annotated extractions file name %s " %(filename))
    df = pd.read_csv(os.path.join(path, filename), index_col=False)
    df = df[df.columns[1:]]
    df = df.fillna("")
    loc_context_counter = Counter()
    temporal_context_counter = Counter()
    loc_context_counter = Counter([l for l in sum(df['locationContext'].apply(lambda x: x.split(",")).to_list(), []) if l != ''])
    temporal_context_counter = Counter([l for l in sum(df['temporalContext'].apply(lambda x: x.split(",")).to_list(), []) if l != ''])
    print(loc_context_counter)
    print(temporal_context_counter)
    loc_context_counters[filename] = loc_context_counter
    temporal_context_counters[filename] = temporal_context_counter
    loc_context_counter.update(temporal_context_counter)
    pd.DataFrame.from_dict(loc_context_counter, orient="index").reset_index().to_excel(writer, sheet_name=filename[:-27], index=False)



**********************************************************************************

The annotated extractions file name sarsdouble_all_linear_order_5_17.xlsx 
Counter({'Hong Kong': 128, 'Inner Mongolia': 39, 'Beijing': 39, 'Guangdong': 24, 'Amoy Gardens': 21, 'China': 20, 'Guangdong Province': 7, 'Shanghai': 6, 'Mainland China': 5, 'Europe': 1, 'Paris': 1})
Counter({' 2003': 304, 'March 17': 135, 'May 10': 135, '1983': 21, '1985': 21, '03/17': 10, '03/20': 10, '03/23': 10, '03/26': 10, '03/29': 10, '04/01': 10, '04/04': 10, '04/07': 10, '04/10': 10, '04/13': 10, '04/16': 10, '04/19': 10, '04/22': 10, '04/25': 10, '04/28': 10, '05/01': 10, '05/04': 10, '05/07': 10, '05/10': 10, 'February 21st': 9, 'February 21': 9, '17 March': 8, '10 May': 8, 'November 2002': 5, 'February 22': 4})

**********************************************************************************

The annotated extractions file name modeling_covid_italy_all_linear_order_5_17.xlsx 
Counter({'Italy': 253, 'Lodi Province'

/usr/local/lib/python3.10/dist-packages/openpyxl/workbook/child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [18]:
df.columns

Index(['doc_id', 'annotated_page_num', 'para_num', 'event_id', 'event',
       'locationContext', 'sentence_text', 'temporalContext', 'explanation',
       'pg_num', 'blk_id', 'sentence_id', 'doc_sentence_count',
       'doc_sent_linear_order', 'linear_order'],
      dtype='object')

In [19]:
from collections import Counter

filenames = ["sarsdouble_all_linear_order_5_17.xlsx", "modeling_covid_italy_all_linear_order_5_17.xlsx", "response-to-covid-19-was-italy-unprepared_all_linear_order_5_17.xlsx"]

loc_tf = {}
# with pd.ExcelWriter(os.path.join(path, "summarized_context_counter.xlsx")) as writer:

for filename in filenames:
  print("\n**********************************************************************************\n")
  print("The annotated extractions file name %s " %(filename))
  df = pd.read_csv(os.path.join(path, filename), index_col=False)
  df = df[df.columns[1:]]
  df = df.fillna("")
  



**********************************************************************************

The annotated extractions file name sarsdouble_all_linear_order_5_17.xlsx 

**********************************************************************************

The annotated extractions file name modeling_covid_italy_all_linear_order_5_17.xlsx 

**********************************************************************************

The annotated extractions file name response-to-covid-19-was-italy-unprepared_all_linear_order_5_17.xlsx 


In [20]:
pd.DataFrame.from_dict(loc_context_counter, orient="index").reset_index()

,index,0
0,Italy,38
1,Lombardy,8
2,Rome,4
3,western European countries,1
4,Spain,1
5,northern regions,1
6,Italian,1
7,Veneto,1
8,Emilia-Romagna,1
9,Piedmont,1


In [21]:
df = df.fillna("")
Counter([l for l in sum(df['locationContext'].apply(lambda x: x.split(",")).to_list(), []) if l != ''])
Counter([l for l in sum(df['temporalContext'].apply(lambda x: x.split(",")).to_list(), []) if l != ''])

Counter({'between 2000 and 2017': 3,
         '2017': 2,
         'pre-COVID-19': 1,
         '2014': 1,
         '12-week period': 2,
         '2020': 3,
         'April 2020': 2,
         'COVID-19 outbreak': 1,
         'COVID-19 pandemic': 1,
         '2019': 2,
         'COVID-19 health crisis': 1,
         'COVID-19 crisis': 5,
         '2005': 2,
         'more recently': 1,
         '2021–2023': 1,
         '31st January 2020': 1,
         'on 3rd February': 1,
         'between 31st January 2020': 1,
         'beginning of June': 1,
         'two months after the beginning of the first wave': 1,
         '1 January–30 April 2015–2019': 1,
         'during the COVID-19 emergency': 1,
         'during the early days of the COVID-19 outbreak': 2,
         'since the early 1990s': 4,
         '2004': 1,
         '2007': 1,
         'early on in the pandemic (March 2020)': 1,
         'between 1st February and 14th April': 2})

### Explore TF-IDF solution and N-GRAM solution since contexts can be phrases

#### Exploring TF-IDF for finding closest context

In [22]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

filenames = ["sarsdouble_all.csv", "modeling_covid_italy_all.csv", "response-to-covid-19-was-italy-unprepared_all.csv"]
events_filenames = {"sarsdouble_all.csv":"sarsdouble_events.csv", "modeling_covid_italy_all.csv":"modeling_covid_italy_events.csv", 
                    "response-to-covid-19-was-italy-unprepared_all.csv":"response_to_covid_19_was_italy_unprepared_events.csv"}

loc_tf = {}
# with pd.ExcelWriter(os.path.join(path, "summarized_context_counter.xlsx")) as writer:

for filename in filenames:
  print("\n**********************************************************************************\n")
  print("The annotated extractions file name %s " %(filename))
  df = pd.read_csv(os.path.join(path, filename), index_col=False)

  sentences = df['sentence_text'].to_list()
  count_vectorizer = CountVectorizer()
  count_vectorizer.fit_transform(sentences)
  print ("Vocabulary: ")
  print(count_vectorizer.vocabulary_)
  vocab = list(count_vectorizer.vocabulary_)
  print(vocab)


  events_df = pd.read_csv(os.path.join(path, events_filenames[filename]), index_col=False)
  events_df.fillna("")
  events_df['locationContext'] = events_df['locationContext'].replace({"^'|'$": ""}, regex=True)
  events_df['temporalContext'] = events_df['temporalContext'].replace({"^'|'$": ""}, regex=True)
  events_df['locationContext'] = events_df['locationContext'].str.replace('nan', "")
  events_df['locationContext'] = events_df['locationContext'].str.replace('\'','')
  events_df['temporalContext'] = events_df['temporalContext'].str.replace('nan', "")
  events_df['temporalContext'] = events_df['temporalContext'].str.replace('\'','')
  
  location_contexts = []
  temporal_contexts = []
  for context in events_df["locationContext"].str.replace('nan', "").tolist():
    context = str(context).replace("nan", "").lower() if type(context) == float else context.lower()
    location_contexts.extend(context.split(",")) 
  for context in events_df["temporalContext"].str.replace('nan', "").tolist():
    context = str(context).replace("nan", "").lower() if type(context) == float else context.lower()
    temporal_contexts.extend(context.split(","))
  location_contexts = list(set(location_contexts))
  temporal_contexts = list(set(temporal_contexts))
  print(len(location_contexts), len(set(location_contexts)))
  print(len(temporal_contexts), len(set(temporal_contexts)))

  freq_term_matrix = count_vectorizer.transform(location_contexts)
  print("Frequency Term matrix")
  print (freq_term_matrix.todense())

  count_array = freq_term_matrix.toarray()
  df = pd.DataFrame(data=count_array, columns=vocab)
  #print(df)

  tfidf = TfidfTransformer(norm="l2")
  tfidf.fit(freq_term_matrix)
  print ("IDF:")
  print(tfidf.idf_)
  freq_term_matrix = count_vectorizer.transform(temporal_contexts)
  print("Frequency Term matrix")
  print (freq_term_matrix.todense())

  count_array = freq_term_matrix.toarray()
  df = pd.DataFrame(data=count_array, columns=vocab)
  #print(df)

  tfidf = TfidfTransformer(norm="l2")
  tfidf.fit(freq_term_matrix)
  print ("IDF:")
  print(tfidf.idf_)
  #print(df.head)
  


**********************************************************************************

The annotated extractions file name sarsdouble_all.csv 
Vocabulary: 
{'background': 181, 'since': 1149, 'november': 855, '2002': 29, 'and': 133, 'perhaps': 916, 'earlier': 431, 'an': 129, 'outbreak': 886, 'of': 867, 'very': 1332, 'contagious': 309, 'atypical': 172, 'pneumonia': 933, 'now': 856, 'named': 831, 'severe': 1125, 'acute': 94, 'respiratory': 1072, 'syndrome': 1222, 'initiated': 696, 'in': 660, 'the': 1239, 'guangdong': 586, 'province': 995, 'china': 254, 'this': 1247, 'started': 1182, 'world': 1374, 'wide': 1362, 'epidemic': 459, 'after': 106, 'medical': 798, 'doctor': 410, 'from': 547, 'guangzhou': 587, 'infected': 683, 'several': 1124, 'persons': 921, 'at': 169, 'hotel': 622, 'kowloon': 734, 'around': 158, 'february': 513, '21st': 32, '2003': 30, 'sar': 1096, 'hong': 617, 'kong': 733, 'although': 125, 'apparently': 142, 'classical': 264, 'its': 727, 'onset': 872, 'pattern': 909, 'became': 1

#### N-Gram solution since contexts can be phrases

In [23]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [24]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.collocations import *

filenames = ["sarsdouble_all.csv", "modeling_covid_italy_all.csv", "response-to-covid-19-was-italy-unprepared_all.csv"]
events_filenames = {"sarsdouble_all.csv":"sarsdouble_events.csv", "modeling_covid_italy_all.csv":"modeling_covid_italy_events.csv", 
                    "response-to-covid-19-was-italy-unprepared_all.csv":"response_to_covid_19_was_italy_unprepared_events.csv"}

loc_tf = {}
# with pd.ExcelWriter(os.path.join(path, "summarized_context_counter.xlsx")) as writer:

for filename in filenames:
  print("\n**********************************************************************************\n")
  print("The annotated extractions file name %s " %(filename))
  df = pd.read_csv(os.path.join(path, filename), index_col=False)

  words = df['sentence_text'].to_list()
  sentences = []
  for sent in df['text'].to_list():
    sentences.extend(sent.split("\n"))

  events_df = pd.read_csv(os.path.join(path, events_filenames[filename]), index_col=False)
  events_df.fillna("")
  events_df['locationContext'] = events_df['locationContext'].replace({"^'|'$": ""}, regex=True)
  # events_df['temporalContext'] = events_df['temporalContext'].replace({"^'|'$": ""}, regex=True)
  events_df['locationContext'] = events_df['locationContext'].str.replace('nan', "")
  events_df['locationContext'] = events_df['locationContext'].str.replace('\'','')
  # events_df['temporalContext'] = events_df['temporalContext'].str.replace('nan', "")
  # events_df['temporalContext'] = events_df['temporalContext'].str.replace('\'','')
  
  location_contexts = []
  temporal_contexts = []
  for context in events_df["locationContext"].str.replace('nan', "").tolist():
    context = str(context).replace("nan", "").lower() if type(context) == float else context.lower()
    location_contexts.extend(context.split(",")) 
  for context in events_df["temporalContext"].str.replace('nan', "").tolist():
    context = str(context).replace("nan", "").lower() if type(context) == float else context.lower()
    temporal_contexts.extend(context.split(","))
  location_contexts = list(set(location_contexts))
  temporal_contexts = list(set(temporal_contexts))
  print(len(location_contexts), len(set(location_contexts)))
  print(len(temporal_contexts), len(set(temporal_contexts)))

  tokens = [word.casefold() for sentence in sentences for sent in sent_tokenize(sentence) for word in word_tokenize(sent)]
  word_fd = nltk.FreqDist(tokens)
  bigram_fd = nltk.FreqDist(nltk.bigrams(tokens))
  finder = BigramCollocationFinder(word_fd, bigram_fd)

  word_fd = nltk.FreqDist(tokens)
  finder = BigramCollocationFinder(word_fd, bigram_fd)
  bigram_measures = nltk.collocations.BigramAssocMeasures()
  scored = finder.score_ngrams(bigram_measures.raw_freq)
  #print(finder.nbest(bigram_measures.pmi, 5))
  #print(scored)

  fourgram_measures = nltk.collocations.QuadgramAssocMeasures()
  finder_4grams = QuadgramCollocationFinder.from_words(tokens)
  scored_4grams = finder_4grams.score_ngrams(fourgram_measures.raw_freq)
  #print(scored_4grams)
  # finder_4grams.apply_word_filter(lambda w: len(w) < 3)


**********************************************************************************

The annotated extractions file name sarsdouble_all.csv 
12 12
31 31

**********************************************************************************

The annotated extractions file name modeling_covid_italy_all.csv 
8 8
28 28

**********************************************************************************

The annotated extractions file name response-to-covid-19-was-italy-unprepared_all.csv 
14 14
29 29


In [25]:
for loc in location_contexts:
  if loc:
    print(loc)
    for t in scored_4grams:
      words, score = t
      text = " ".join(words)
      if loc in text:
        print(loc, text )

western european countries
western european countries most western european countries
western european countries western european countries ;
western european countries all western european countries
western european countries western european countries (
lombardy
lombardy lombardy , veneto ,
lombardy of lombardy , veneto
lombardy authorities in lombardy in
lombardy health authorities in lombardy
lombardy in lombardy in codogno
lombardy lombardy in codogno (
lombardy the whole of lombardy
lombardy whole of lombardy ,
lombardy ( lombardy region ,
lombardy , especially in lombardy
lombardy . in lombardy ,
lombardy . lombardy , the
lombardy especially in lombardy .
lombardy especially in the lombardy
lombardy governments of lombardy ,
lombardy homes . in lombardy
lombardy homes in lombardy (
lombardy in lazio . lombardy
lombardy in lombardy ( serra
lombardy in lombardy , for
lombardy in lombardy . this
lombardy in the lombardy region
lombardy isolated ( lombardy region
lombardy lazio . lo

In [26]:
temporal_contexts

['',
 'pre-covid-19',
 'during the early days of the covid-19 outbreak',
 '2004',
 'two months after the beginning of the first wave',
 'covid-19 pandemic',
 'between 2000 and 2017',
 '2021–2023',
 'during the covid-19 emergency',
 'more recently',
 'on 3rd february',
 'covid-19 health crisis',
 'since the early 1990s',
 'between 31st january 2020',
 '2014',
 'covid-19 outbreak',
 'between 1st february and 14th april',
 '2017',
 'covid-19 crisis',
 '1 january–30 april 2015–2019',
 '2020',
 'april 2020',
 '12-week period',
 '31st january 2020',
 '2005',
 'early on in the pandemic (march 2020)',
 '2007',
 '2019',
 'beginning of june']

In [27]:
location_contexts

['',
 'western european countries',
 'lombardy',
 'veneto',
 'spain',
 'northern regions',
 'italy',
 'european countries and the usa',
 'italian regions',
 'emilia-romagna',
 'northern italy',
 'piedmont',
 'rome',
 'italian']

In [28]:
len(tfidf.idf_), len(vocab)

(1803, 1803)

In [29]:
for ctx in location_contexts:
  if ctx in count_vectorizer.vocabulary_.keys():
    print(ctx,count_vectorizer.vocabulary_[ctx])
  else:
    print(ctx, " does not exist in this vocab")

  does not exist in this vocab
western european countries  does not exist in this vocab
lombardy 1013
veneto 1746
spain 1538
northern regions  does not exist in this vocab
italy 941
european countries and the usa  does not exist in this vocab
italian regions  does not exist in this vocab
emilia-romagna  does not exist in this vocab
northern italy  does not exist in this vocab
piedmont 1228
rome 1454
italian 939


In [30]:
if "italy" in count_vectorizer.vocabulary_.keys():
  print("Italy")

Italy


##### numpy way of sorting multiple columns

In [31]:
a = np.array([[9, 2, 3],
           [4, 5, 6],
           [7, 0, 5]])
a1 = a[a[:, 0].argsort()]
a2 = a1[a1[:, 1].argsort()]
a3 = a2[a2[:, 2].argsort()]
a1,a2,a3

(array([[4, 5, 6],
        [7, 0, 5],
        [9, 2, 3]]),
 array([[7, 0, 5],
        [9, 2, 3],
        [4, 5, 6]]),
 array([[9, 2, 3],
        [7, 0, 5],
        [4, 5, 6]]))

## Solution: for searching the contexts
- For each pdf in pdf_names:
  - First construct contexts dictionary with contexts as keys and doc_id, block_id, sentence_number, **all extractions' linear_order_number** from {this_pdf_name}_events.xlsx
  - Get doc_sentence_map with doc_id, sentence_text, sentence_id, doc_sent_linear_order from {this_pdf_name}_all.csv
  - For each context key in contexts dictionary :
    - this_doc_id, sentence_number from contexts dictionary
    - from doc_sentence_map, get all sentences up until **doc_sent_map's doc_sent_linear_order**
    - get 3 top closest doc_id, sentence_number pairs where the context was found up until this_doc_id,this_sentence_number
    - calculate distance between **doc_sent_map's doc_sent_linear_order** between context dictionary value and doc_sentence_map for each of the nearest context matches
- Plot the histogram

In [38]:
def get_distances(event_values:[], df:pd.DataFrame,  ctx:str, contexts_results, count):
  results = []
  event_ctx = []
  for val in event_values:
    doc_id = val[0]
    sentence_num, doc_sent_linear_order, linear_order = val[6:]
    #print(doc_id, sentence_num, doc_sent_linear_order, linear_order)
    sentences = df[df['sno']<=doc_sent_linear_order][['doc_id','sentence_id','sentence_text', 'sno']].values.tolist()
    distances = {}
    for sentence in sentences:
      #print(ctx, sentence[2])
      if ctx != "NONE" and ctx in sentence[2].lower():
        #print("context "+ctx,sentence[2], doc_sent_linear_order, sentence[3], doc_sent_linear_order - sentence[3])
        distances[sentence[3]] = doc_sent_linear_order - sentence[3]
        if doc_sent_linear_order - sentence[3] == 0:
          count += 1

    sorted_distances = dict(sorted(distances.items(), key=lambda x:x[1]))
    top_3 = nsmallest(3, sorted_distances, key=sorted_distances.get)
    if len(top_3)>0:
      print(top_3[0], sorted_distances[top_3[0]] )
      dist = sorted_distances[top_3[0]]
    else:
      top_3.append( -1)
      dist = -1
    contexts_results[ctx] = {'doc_id':val[0], 'event_id':val[1], 'context':ctx,
    'event':val[2], 'sentence_text':val[3], 'pg_num':val[4], 'blk_id':val[5],
    "sentence_id":sentence_num, "doc_sent_linear_order":doc_sent_linear_order, "linear_order":linear_order, "distance": sorted_distances, 
    "top": dist, "sno":top_3[0]}
    event_ctx.append({'event_id':val[1], "ctx":ctx, "dist":dist})
    results.append({"ctx":ctx,"doc_sent_linear_order":doc_sent_linear_order, "linear_order":linear_order,"dist": dist, "sno":top_3[0]})
  return contexts_results, count, results, event_ctx

In [39]:
filenames = ["sarsdouble_all.csv", "modeling_covid_italy_all.csv", "response-to-covid-19-was-italy-unprepared_all.csv"]
events_filenames = {"sarsdouble_all.csv":"sarsdouble_events.csv", "modeling_covid_italy_all.csv":"modeling_covid_italy_events.csv", 
                    "response-to-covid-19-was-italy-unprepared_all.csv":"response_to_covid_19_was_italy_unprepared_events.csv"}

loc_tf = {}
# with pd.ExcelWriter(os.path.join(path, "summarized_context_counter.xlsx")) as writer:
from heapq import nsmallest

print("\n**********************************************************************************\n")
print("The annotated extractions file name %s " %(filenames[2]))
df = pd.read_csv(os.path.join(path, filenames[2]), index_col=False)
count = 0
words = df['text'].to_list()
sentences = []
for sent in df['text'].to_list():
  sentences.extend(sent.split("\n"))


**********************************************************************************

The annotated extractions file name response-to-covid-19-was-italy-unprepared_all.csv 


In [40]:
all_results = {filename: {} for filename in filenames}

for filename in filenames:
  print("\n**********************************************************************************\n")
  print("The annotated extractions file name %s " %(filename))
  df = pd.read_csv(os.path.join(path, filename), index_col=False)

  words = df['sentence_text'].to_list()
  sentences = []
  for sent in df['text'].to_list():
    sentences.extend(sent.split("\n"))

  events_df = pd.read_csv(os.path.join(path, events_filenames[filename]), index_col=False)
  events_df.fillna("")
  events_df['locationContext'] = events_df['locationContext'].replace({"^'|'$": ""}, regex=True)
  # events_df['temporalContext'] = events_df['temporalContext'].replace({"^'|'$": ""}, regex=True)
  events_df['locationContext'] = events_df['locationContext'].str.replace('nan', "")
  events_df['locationContext'] = events_df['locationContext'].str.replace('\'','')
  # events_df['temporalContext'] = events_df['temporalContext'].str.replace('nan', "")
  # events_df['temporalContext'] = events_df['temporalContext'].str.replace('\'','')
  
  events_lctx = []
  events_tctx = []
  location_contexts = []
  temporal_contexts = []
  events_df["locationContext"] = events_df["locationContext"].apply(lambda x: str(x).replace("nan", "NONE").lower()
                                if type(x) == float else x.lower())
  events_df["temporalContext"] = events_df["temporalContext"].apply(lambda x: str(x).replace("nan", "NONE").lower()
                                if type(x) == float else x.lower())
  events_df["location_contexts"] = events_df["locationContext"].apply(lambda x: str(x).replace("nan", "NONE").lower().split(",") 
                                if type(x) == float else x.lower().split(","))
  events_df["temporal_contexts"] = events_df["temporalContext"].apply(lambda x: str(x).replace("nan", "NONE").lower().split(",") 
                                if type(x) == float else x.lower().split(","))
  for context in events_df["location_contexts"].tolist():
    if context != "NONE" :
      location_contexts.extend(context) 
  for context in events_df["temporal_contexts"].tolist():
    if context != "NONE" :
      temporal_contexts.extend(context)
  ulocation_contexts = list(set(location_contexts))
  utemporal_contexts = list(set(temporal_contexts))
  print(len(location_contexts), len(set(location_contexts)))
  print(len(temporal_contexts), len(set(temporal_contexts)))
  all_contexts = ulocation_contexts + utemporal_contexts
  lcontexts = {}
  tcontexts={}
  for ctx in ulocation_contexts:
    lcontexts[ctx] = events_df[events_df["locationContext"].str.contains(ctx)][['doc_id', 'event_id',
       'event', 'sentence_text', 'pg_num', 'blk_id', 'sentence_id', 'doc_sent_linear_order', 'linear_order']].values.tolist()
  for ctx in utemporal_contexts:
    tcontexts[ctx] = events_df[events_df["temporalContext"].str.contains(ctx)][['doc_id', 'event_id',
       'event', 'sentence_text', 'pg_num', 'blk_id', 'sentence_id', 'doc_sent_linear_order', 'linear_order']].values.tolist()
  lcontexts_results = {}
  tcontexts_results = {}
  ldist_res = []
  tdist_res = []
  for ctx, values in lcontexts.items():
    lcontexts_results[ctx] = []
    #print("context ", ctx)
    contexts_results, count, results, event_ctx = get_distances(values, df, ctx, lcontexts_results, count)
    lcontexts_results[ctx] = contexts_results
    ldist_res.extend(results)
    events_lctx.extend(event_ctx)
  for ctx, values in tcontexts.items():
    tcontexts_results[ctx] = []
    contexts_results, count, results, event_ctx = get_distances(values, df, ctx, tcontexts_results, count)
    tcontexts_results[ctx] = contexts_results
    tdist_res.extend(results)
    events_tctx.extend(event_ctx)
  all_results[filename] = {"loc_results":ldist_res, "temp_results": tdist_res, "l_event_ctx":events_lctx, "t_event_ctx":events_tctx}


**********************************************************************************

The annotated extractions file name sarsdouble_all.csv 
295 12
853 31
13 43
13 46
13 46
13 46
117 17
117 3
261 14
245 1
245 1
245 1
245 1
245 1
245 1
245 1
245 1
245 1
245 1
245 1
245 1
245 1
245 1
245 1
245 3
261 17
261 24
117 63
117 64
117 65
117 75
117 75
117 75
193 2
193 5
193 5
193 5
193 5
193 5
200 10
22 48
22 50
22 53
22 55
22 10
79 141
79 4
79 9
67 3
67 5
75 0
75 2
1 4
1 4
79 58
79 51
79 26
79 15
79 16
79 16
79 16
79 18
79 18
79 19
79 19
79 19
79 77
79 77
205 22
205 22
228 6
228 7
228 9
261 2
261 2
159 29
159 29
31 39
31 41
31 44
31 46
31 34
31 34
31 8
31 1
31 3
112 25
2 13
205 60
205 60
205 89
205 89
205 92
205 92
205 92
205 93
205 93
205 94
205 15
53 17
53 19
53 22
53 24
53 3
53 6
53 6
53 6
22 10
79 41
261 14
261 24
304 0
305 1
305 1
305 1
305 1
305 1
305 1
305 1
305 1
305 1
305 1
305 1
305 3
305 3
305 3
305 3
305 4
79 101
79 102
79 103
193 5
193 5
193 5
193 5
193 5
200 10
218 2
153 0
106 9
1

<ipython-input-40-33f35723a2d2>:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tcontexts[ctx] = events_df[events_df["temporalContext"].str.contains(ctx)][['doc_id', 'event_id',
<ipython-input-40-33f35723a2d2>:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tcontexts[ctx] = events_df[events_df["temporalContext"].str.contains(ctx)][['doc_id', 'event_id',
<ipython-input-40-33f35723a2d2>:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tcontexts[ctx] = events_df[events_df["temporalContext"].str.contains(ctx)][['doc_id', 'event_id',


 124
47 124
47 127
47 116
47 117
47 117
47 117
47 117
47 117
47 117
47 119
47 119
47 119
47 119
47 119
47 119
47 109
47 109
47 112
47 104
47 107
378 1
378 20
378 20
378 20
422 1
422 1
422 1
422 1
422 1
422 1
422 5
422 5
422 8
433 0
433 3
433 3
433 3
433 3
433 4
433 5
441 6
441 6
441 8
441 11
441 12
441 17
441 17
441 17
441 17
441 17
441 17
441 17
441 17
441 17
441 17
441 17
441 17
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 18
441 19
441 19
441 19
441 20
441 20
441 20
441 20
441 20
441 21
441 21
441 22
441 23
441 23
441 23
441 23
441 23
441 23
441 23
441 23
441 23
441 23
441 23
441 23
441 24
441 25
441 25
441 25
441 26
441 26
441 26
441 26
441 26
441 26
441 27
441 31
441 31
441 31
441 39
441 39
441 40
441 40
441 42
441 42
441 43


<ipython-input-40-33f35723a2d2>:51: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  tcontexts[ctx] = events_df[events_df["temporalContext"].str.contains(ctx)][['doc_id', 'event_id',


16
242 18
141 16
174 0
174 0
212 0
7 2
77 1
115 0
115 7
115 9
79 14
96 1
100 0
100 3
62 16
79 0
79 4
62 11
62 12
44 17
62 0
5 0
5 0
8 1
8 1
8 1
237 18
258 0
260 0
147 0
147 10
171 0
173 1
173 7
182 0
182 0
182 3
182 6
182 7
192 0
192 0
192 2
192 3
201 6
210 2
258 0
258 2
255 0
182 0
182 0
13 65
13 60
171 0
255 0
153 0
9 0
182 0
182 0
194 0
174 0
48 31
5 0
5 0
5 4
178 2
122 0
157 0
93 0
188 0
188 1
188 4
188 4
127 128
171 0
9 0
77 1
182 0
182 0
89 169
89 171
5 0
5 0
5 4
71 26
100 0
100 3
147 0
199 13
89 4
89 8
73 0
73 1
61 0
153 0
171 0
207 0
73 0
73 1
62 0
153 0
171 0
115 0
115 9
195 0
83 0
174 0
185 0
171 0


In [42]:
t = pd.DataFrame.from_records(all_results['sarsdouble_all.csv']['t_event_ctx'])
t.drop(t.index[t['dist'] == -1], inplace = True)

In [44]:
t.groupby('dist').count().reset_index().to_excel("sarsdouble_event_ctx_distance_counts.xlsx")

In [46]:
filename[:-8]

'response-to-covid-19-was-italy-unprepared'

In [49]:
for filename in all_results.keys():
  with pd.ExcelWriter(os.path.join(path, filename[:-8]+"_events_context_distances.xlsx")) as writer:

    t = pd.DataFrame.from_records(all_results['sarsdouble_all.csv']['t_event_ctx'])
    t.drop(t.index[t['dist'] == -1], inplace = True)

    t.to_excel(writer, sheet_name="temporal", index=False)
    t.groupby('dist').count().reset_index().to_excel(writer, sheet_name="t_summary_counts", index=False)

    t = pd.DataFrame.from_records(all_results[filename]['l_event_ctx'])
    t.drop(t.index[t['dist'] == -1], inplace = True)

    t.to_excel(writer, sheet_name="location", index=False)
    t.groupby('dist').count().reset_index().to_excel(writer, sheet_name="l_summary_counts", index=False)

In [ ]:
with pd.ExcelWriter(os.path.join(path, "summarized_context__distance_counter.xlsx")) as writer:

  for filename in all_results.keys():
    print("\n**********************************************************************************\n")
    print("The annotated extractions file name %s " %(filename))
    df = pd.read_csv(os.path.join(path, filename), index_col=False)
    df = df[df.columns[1:]]
    df = df.fillna("")
    loc_context_counter = Counter()
    temporal_context_counter = Counter()
    loc_context_counter = Counter([l for l in sum(df['locationContext'].apply(lambda x: x.split(",")).to_list(), []) if l != ''])
    temporal_context_counter = Counter([l for l in sum(df['temporalContext'].apply(lambda x: x.split(",")).to_list(), []) if l != ''])
    print(loc_context_counter)
    print(temporal_context_counter)
    loc_context_counters[filename] = loc_context_counter
    temporal_context_counters[filename] = temporal_context_counter
    loc_context_counter.update(temporal_context_counter)
    pd.DataFrame.from_dict(loc_context_counter, orient="index").reset_index().to_excel(writer, sheet_name=filename[:-27], index=False)

In [ ]:
import altair as alt

In [ ]:
res = pd.DataFrame.from_dict(all_results['sarsdouble_all.csv']['loc_results'])
res.drop(res.index[res['top'] == -1], inplace = True)
r = res.groupby(['top']).count().reset_index()[['sno', 'top']]
ch = alt.Chart(r).mark_bar().encode(
    alt.X("top:Q", title="Nearest Distances"),
    y=alt.Y('sno',title="Count #")
).properties(
    title=alt.TitleParams('Sarsdouble Paper: Histogram for Location context distances',
    subtitle=["linear order and distances of nearest contexts","Event Extractions"]), width=800
)

ch.configure_title(
    fontSize=14,
    font='Courier',
    anchor='start',
    color='gray'
)
ch.mark_bar() + ch.mark_text(align='left', dy=2)


alt.LayerChart(...)

In [ ]:
# (e1, c1) - 3
# (e2, c2) - 13
# (e3, c3) - 2
# Bin1: [0, 10] => 2
# Bin2: [10, 20] => 1
example:  event and context - 91 
91 - # of 
0,9,2,6,40,91 -> dont divide bins 

In [ ]:
res.groupby(['top']).count().reset_index()[['sno', 'top']]

,sno,top
0,26,-1
1,34,0
2,41,1
3,15,2
4,21,3
5,9,4
6,17,5
7,11,6
8,4,7
9,2,8


In [ ]:
t = pd.DataFrame.from_dict(all_results['sarsdouble_all.csv']['temp_results']).groupby(['top']).count().reset_index()[['sno', 'top']]
t = t.rename(columns={'sno':"Count #", "top":"Distances"})
t.drop(t.index[t['Distances'] == -1], inplace = True)
t[t['Distances'] >=80]

,Count #,Distances
49,40,80
50,2,103
51,2,135
52,1,136
53,4,225


In [ ]:
t[t['Distances'] <=80]

,Count #,Distances
1,3,0
2,1,1
3,6,2
4,6,3
5,5,4
6,8,5
7,18,6
8,3,7
9,9,8
10,2,9


In [ ]:
t = pd.DataFrame.from_dict(all_results['sarsdouble_all.csv']['temp_results'])
t.drop(t.index[t['top'] == -1], inplace = True)
t[t['top']==-1]

,ctx,doc_sent_linear_order,linear_order,top,sno


In [ ]:
res = pd.DataFrame.from_dict(all_results['sarsdouble_all.csv']['temp_results'])
res.drop(res.index[res['top'] == -1], inplace = True)
r = res.groupby(['top']).count().reset_index()[['sno', 'top']]
ch = alt.Chart(r).mark_bar(size=40).encode(
    alt.X("top:Q", title="Nearest Distances"),
    y=alt.Y('sno',title="Count #"), color="sno:N"
).properties(
        title=alt.TitleParams('Histogram for Temporal context distances: sarsdouble',
    subtitle=["linear order and distances of nearest contexts","Event Extractions"]), width=800
)
ch.configure_title(
    fontSize=14,
    font='Courier',
    anchor='start',
    color='gray'
)
ch + ch.mark_text(align='left', dy=2)


alt.LayerChart(...)

In [ ]:
res = pd.DataFrame.from_dict(all_results['sarsdouble_all.csv']['temp_results'])
res.drop(res.index[res['top'] == -1], inplace = True)
r = res.groupby(['top']).count().reset_index()[['sno', 'top']]
ch = alt.Chart(r).mark_bar(size=20).encode(
    alt.X("top:Q", title="Nearest Distances"),
    y=alt.Y('sno',title="Count #")
).properties(
        title=alt.TitleParams('Sarsdouble Paper: Histogram for Temporal context distances',
    subtitle=["linear order and distances of nearest contexts","Event Extractions"]), width=800
)
ch.configure_title(
    fontSize=14,
    font='Courier',
    anchor='start',
    color='gray'
)
ch + ch.mark_text(align='left', dy=2)


alt.LayerChart(...)

In [ ]:
res = pd.DataFrame.from_dict(all_results['modeling_covid_italy_all.csv']['loc_results'])
res.drop(res.index[res['top'] == -1], inplace = True)
r = res.groupby(['top']).count().reset_index()[['sno', 'top']]
ch = alt.Chart(r).mark_bar(size=40).encode(
    alt.X("top:Q", title="Nearest Distances"),
    y=alt.Y('sno:Q',title="Count #")
).properties(
        title=alt.TitleParams('Modeling_covid_italy Paper: Histogram for Location context distances',
    subtitle=["linear order and distances of nearest contexts","Event Extractions"]), width=1000
)

ch.configure_title(
    fontSize=14,
    font='Courier',
    anchor='start',
    color='gray'
)
ch.mark_bar() + ch.mark_text(align='left', dy=2)


alt.LayerChart(...)

In [ ]:
res = pd.DataFrame.from_dict(all_results['modeling_covid_italy_all.csv']['temp_results'])
res.drop(res.index[res['top'] == -1], inplace = True)
r = res.groupby(['top']).count().reset_index()[['sno', 'top']]
ch = alt.Chart(r).mark_bar(size=40).encode(
    alt.X("top:Q", title="Nearest Distances"),
    y=alt.Y('sno:Q',title="Count #")
).properties(
    title=alt.TitleParams('Modeling_covid_italy Paper: Histogram for Temporal context distances',
    subtitle=["linear order and distances of nearest contexts","Event Extractions"]), width=800
)

ch.configure_title(
    fontSize=14,
    font='Courier',
    anchor='start',
    color='gray'
)
# ch + ch.mark_text(
#     dy=3, # update
#     color='black'
# ).encode(
#     y=alt.value(0), # update
#     text='sno:Q',
# )
ch

alt.Chart(...)

In [ ]:
res = pd.DataFrame.from_dict(all_results['response-to-covid-19-was-italy-unprepared_all.csv'] ['loc_results'])
res.drop(res.index[res['top'] == -1], inplace = True)
r = res.groupby(['top']).count().reset_index()[['sno', 'top']]
ch = alt.Chart(r).mark_bar(size=30).encode(
    alt.X("top:Q", title="Nearest Distances"),
    y=alt.Y('sno',title="Count #")
).properties(
    title=alt.TitleParams('Response-to-covid-19-was-italy-unprepared: Histogram for Location context distances',
    subtitle=["linear order and distances of nearest contexts","Event Extractions"]), width=800
)

ch.configure_title(
    fontSize=14,
    font='Courier',
    anchor='start',
    color='gray'
)
ch.mark_bar() + ch.mark_text(align='left', dy=2)


alt.LayerChart(...)

In [ ]:
res = pd.DataFrame.from_dict(all_results['response-to-covid-19-was-italy-unprepared_all.csv']['temp_results'])
res.drop(res.index[res['top'] == -1], inplace = True)
r = res.groupby(['top']).count().reset_index()[['sno', 'top']]
ch = alt.Chart(r).mark_bar(size=40).encode(
    alt.X("top:Q", title="Nearest Distances"),
    y=alt.Y('sno',title="Count #")
).properties(
    title=alt.TitleParams('Response-to-covid-19-was-italy-unprepared Paper: Histogram for Temporal context distances',
    subtitle=["linear order and distances of nearest contexts","Event Extractions"]), width=800
)

ch.configure_title(
    fontSize=14,
    font='Courier',
    anchor='start',
    color='gray'
)
ch.mark_bar() + ch.mark_text(align='left', dy=2)


alt.LayerChart(...)

In [ ]:
all_results['sarsdouble_all.csv']['loc_results']

[{'ctx': 'amoy gardens',
  'doc_sent_linear_order': 41,
  'linear_order': 822,
  'top': -1,
  'sno': -1},
 {'ctx': 'amoy gardens',
  'doc_sent_linear_order': 41,
  'linear_order': 824,
  'top': -1,
  'sno': -1},
 {'ctx': 'amoy gardens',
  'doc_sent_linear_order': 43,
  'linear_order': 867,
  'top': -1,
  'sno': -1},
 {'ctx': 'amoy gardens',
  'doc_sent_linear_order': 43,
  'linear_order': 868,
  'top': -1,
  'sno': -1},
 {'ctx': 'amoy gardens',
  'doc_sent_linear_order': 46,
  'linear_order': 911,
  'top': -1,
  'sno': -1},
 {'ctx': 'amoy gardens',
  'doc_sent_linear_order': 46,
  'linear_order': 912,
  'top': -1,
  'sno': -1},
 {'ctx': 'amoy gardens',
  'doc_sent_linear_order': 46,
  'linear_order': 913,
  'top': -1,
  'sno': -1},
 {'ctx': 'amoy gardens',
  'doc_sent_linear_order': 46,
  'linear_order': 914,
  'top': -1,
  'sno': -1},
 {'ctx': 'amoy gardens',
  'doc_sent_linear_order': 46,
  'linear_order': 915,
  'top': -1,
  'sno': -1},
 {'ctx': 'amoy gardens',
  'doc_sent_linear_or

In [ ]:
events_df["location_contexts"] = events_df["locationContext"].apply(lambda x: str(x).replace("nan", "").lower().split(",") 
                                if type(x) == float else x.lower().split(","))
events_df["locationContext"] = events_df["locationContext"].apply(lambda x: str(x).replace("nan", "").lower()
                                if type(x) == float else x.lower())

In [ ]:
Counter(location_contexts)

Counter({'italy': 38,
         'northern regions': 1,
         'rome': 4,
         'lombardy': 8,
         'none': 2,
         'western european countries': 1,
         'spain': 1,
         'italian regions': 1,
         'european countries and the usa': 2,
         'italian': 1,
         'veneto': 1,
         'emilia-romagna': 1,
         'piedmont': 1,
         'northern italy': 2})

In [ ]:
for filename in filenames:
  print("\n**********************************************************************************\n")
  print("The annotated extractions file name %s " %(filename))
  df = pd.read_csv(os.path.join(path, filename), index_col=False)

  words = df['sentence_text'].to_list()
  sentences = []
  for sent in df['text'].to_list():
    sentences.extend(sent.split("\n"))

  events_df = pd.read_csv(os.path.join(path, events_filenames[filename]), index_col=False)
  events_df.fillna("")
  events_df['locationContext'] = events_df['locationContext'].replace({"^'|'$": ""}, regex=True)
  # events_df['temporalContext'] = events_df['temporalContext'].replace({"^'|'$": ""}, regex=True)
  events_df['locationContext'] = events_df['locationContext'].str.replace('nan', "")
  events_df['locationContext'] = events_df['locationContext'].str.replace('\'','')
  # events_df['temporalContext'] = events_df['temporalContext'].str.replace('nan', "")
  # events_df['temporalContext'] = events_df['temporalContext'].str.replace('\'','')
  

##### ignore

In [ ]:
path = "/content/drive/MyDrive/Colab Notebooks/skema/data/"
extractions_path = "cosmos-and-extractions-jsons-for-3-papers"
annotated_files = ["data-sars-double.json", "data_modeling_covid_italy.json", "data-response-to-covid-19-was-italy-unprepared.json"]
extraction_files =["extractions_sarsdouble.json", "extractions_modeling_covid_italy--COSMOS-data.json","extractions_response-to-covid-19-was-italy-unprepared--COSMOS-data.json"]
paper_names = ["sarsdouble.pdf", "modeling_covid_italy.pdf", "response-to-covid-19-was-italy-unprepared.pdf"]
ann_extr_file_pairs = {}
for name,ann, extr in zip(paper_names,annotated_files, extraction_files):
  ann_extr_file_pairs[name] = [ann, extr]

In [ ]:
ann_extr_file_pairs

In [ ]:
for ann, extr in list(ann_extr_file_pairs.values())[:1]:
  print("This extractions file ",extr)
  with open(os.path.join(path,extractions_path, extr ), "r", encoding='UTF-8') as f:
    contents = f.readlines()
    extractions = json.loads(contents[0])
    

In [ ]:
# events

# eventId
# page number , block number, sentence id -page -> linear order

# eventid 1,2,3,4,5

# linear sentence_order : 6,10,12,12,14

# location context
# temporal context 

# 1) missing linear order in TextReadingPipeline output -> extractions json
# 2) location context - annotations 
# 	for each eventID in eventMentions in annotations

# 		locations = [ "Italy", "Rome"]
# 		linear order entire extractions -> linear order for this evntID
# 			sentenceID -> documents and look in sentence text from this sentenceID 
# 			and get closest sentenceID where you find this location's text